Шаг 0. Слияние данных

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import os

merged_data = pd.DataFrame()

#years = list(range(2012, 2018))
years = [2011, 2019]
articles = ['105', '107', '111_4']

# для анализа не разделяем по статьям
for year in years:
    data_gas_year = pd.DataFrame()
    if year == 2018:
        continue
    for art in articles:
        data_gas = pd.read_excel(f'Данные_за_все_года/{art}_{year}_cases_texts.xlsx', index_col=0)
        data_gas['year'] = year
        data_gas['source'] = 'ГАС Правосудие'
        data_gas['article'] = art
        data_gas = data_gas[['Субъект РФ', 'ID', 'text', 'year', 'source']]
        data_gas.columns = ['region', 'ID', 'text', 'year', 'source']
        data_gas_year = pd.concat([data_gas_year, data_gas])
    data_mgs = pd.read_excel(f'Данные_за_все_года/MOSGORSUD_texts_{year}.xlsx', index_col=0)
    data_mgs['year'] = year
    data_mgs['source'] = 'Мосгорсуд'
    data_mgs['Субъект РФ'] = 'Москва'
    data_mgs = data_mgs[['Субъект РФ', 'custom_id', 'text', 'year', 'source']]
    data_mgs.columns = ['region', 'ID', 'text', 'year', 'source']
    merged_data = pd.concat([merged_data, data_gas_year, data_mgs])
    merged_data = merged_data.drop_duplicates('ID')
    merged_data = merged_data.set_index('ID')
    merged_data.to_csv(f'Данные_за_все_года/{year}_merged.csv')

Шаг 1. Отбор только женских дел

In [4]:
import re

patterns = [
    r'скончалась',
    r'погибш[аеу][йяю]',
    r'смерт[иь]ю? потерпевшей',
    r'смерт[иь]ю? последней',
    r'мертва',
    r'е[её] труп',
    r'труп женщины'
]

def check_patterns(text):
    global patterns
    
    for pattern in patterns:
        if re.search(pattern, text):
            return True
    return False

In [6]:
#years = list(range(2012, 2018))
#years = [2012]

women_cases_by_year = []

for year in years:
    data = pd.read_csv(f'Данные_за_все_года/{year}_merged.csv')
    data = data[(data['ID'].notna())&(data['text'].notna())].set_index('ID')
    women_cases = data[data['text'].apply(check_patterns)]
    women_cases_by_year.append([year, len(women_cases)])
    print(year, '-', len(women_cases))
    women_cases.to_csv(f'Данные_за_все_года/{year}_women.csv')
res = pd.DataFrame(women_cases_by_year, columns=['год', 'число женских дел'])
display(res)
#res.to_excel('women_cases_dynamics.xlsx')
    #display(women_cases.head(), len(women_cases))
    #print(len(women_cases))

2011 - 2075
2019 - 1419


,год,число женских дел
0,2011,2075
1,2019,1419


#### Шаг 2. Предобработка текстов

In [7]:
import re
def keep_only_rus(text):    
    new_text = ''
    for symbol in text:
        if re.match(r'[А-я]', symbol) or symbol == ' ':
            new_text += symbol
        else:
            new_text += ' '
    return new_text

def del_double_spaces(text_with_double_spaces):
    while '  ' in text_with_double_spaces:
        text_with_double_spaces = text_with_double_spaces.replace('  ',' ')
    return text_with_double_spaces

import pymystem3
mstem = pymystem3.Mystem()

def lemmatize(raw_text):
    return ''.join(mstem.lemmatize(raw_text)).strip()

import stop_words

stopwords = stop_words.get_stop_words('russian')
stopwords.extend(stop_words.get_stop_words('english'))
stopwords = list(set(stopwords))
stopwords += [
    'фио', 'гггг', 'подсудимый', 'суд',
    'изымать', 'согласно', 'наказание',
    'потерпевший', 'показание', 'судебный',
    'преступление', 'адрес', 'свидетель',
    'свой', 'находиться', 'час', 'ход'
             ]

def del_stopwords(text):
    global stopwords
    new_text = []
    for word in text.split():
        if word not in stopwords and len(word) > 2:
            new_text.append(word)
    return ' '.join(new_text)

In [8]:
#years = list(range(2012, 2018))
#years = [2012]
for year in years:
    print(year)
    data = pd.read_csv(f'Данные_за_все_года/{year}_women.csv', index_col=0)
    data['text_prep'] = data['text'].str.lower()
    data['text_prep'] = data['text_prep'].apply(keep_only_rus)
    data['text_prep'] = data['text_prep'].apply(del_double_spaces)
    data['text_prep'] = data['text_prep'].apply(lemmatize)
    data['text_prep'] = data['text_prep'].apply(del_stopwords)
    data['text_prep'] = data['text_prep'].apply(del_double_spaces)
    data = data[data['text_prep'].notna()]
    data.to_csv(f'Данные_за_все_года/{year}_women_prep.csv')

2011
2019


#### Шаг 3. Применение моделей

In [9]:
import pickle

with open("cvect.pkl", 'rb') as file:
    cvect = pickle.load(file)
    
with open("gbc_dv.pkl", 'rb') as file:
    dv = pickle.load(file)
    
with open("gbc_ipv.pkl", 'rb') as file:
    ipv = pickle.load(file)

In [10]:
#years = list(range(2012, 2018))
#years = [2012]

prec_dv_model = 0.86
prec_ipv_model = 0.94

dv_by_years = []

for year in years:
    print(year)
    data = pd.read_csv(f'Данные_за_все_года/{year}_women_prep.csv', index_col=0)
    matrix = cvect.transform(data['text_prep'])
    td_matrix = pd.DataFrame(matrix.toarray(), index=data.index, columns=cvect.get_feature_names())
    td_matrix['DV'] = dv.predict(td_matrix)
    dv_preds = td_matrix[['DV']]
    td_matrix_ipv = td_matrix[td_matrix['DV']==1].drop('DV', axis=1)
    td_matrix_ipv['IPV'] = ipv.predict(td_matrix_ipv)
    dv_preds['IPV'] = td_matrix_ipv['IPV']
    dv_by_years.append([
        year, 
        len(data), 
        dv_preds['DV'].sum(),
        dv_preds['DV'].sum()*prec_dv_model,
        dv_preds['IPV'].sum(),
        dv_preds['IPV'].sum()*prec_dv_model*prec_ipv_model
    ])
    dv_preds.to_excel(f'Данные_за_все_года/{year}_women_predictions.xlsx')
dv_by_years_final = pd.DataFrame(dv_by_years, columns = [
    'год', 'число женских дел', 'из них ДН', 'из них ДН (с поправкой на точность)',
    'из них партнерское', 'из них партнерское (с поправкой на точность)'
])
display(dv_by_years_final)

2011
2019


,год,число женских дел,из них ДН,из них ДН (с поправкой на точность),из них партнерское,из них партнерское (с поправкой на точность)
0,2011,2075,1578,1357.08,1356.0,1096.1904
1,2019,1419,1108,952.88,965.0,780.1060


In [11]:
dv_by_years_final['доля ДН (%)'] = dv_by_years_final['из них ДН (с поправкой на точность)'] / dv_by_years_final['число женских дел'] * 100
dv_by_years_final['доля партнерского из ДН (%)'] = dv_by_years_final['из них партнерское (с поправкой на точность)'] / dv_by_years_final['из них ДН (с поправкой на точность)'] * 100
dv_by_years_final

,год,число женских дел,из них ДН,из них ДН (с поправкой на точность),из них партнерское,из них партнерское (с поправкой на точность),доля ДН (%),доля партнерского из ДН (%)
0,2011,2075,1578,1357.08,1356.0,1096.1904,65.401446,80.775665
1,2019,1419,1108,952.88,965.0,780.1060,67.151515,81.868231


In [12]:
dv_by_years_final['доля ДН (нижняя граница)'] = 100*(dv_by_years_final['доля ДН (%)']/100 - 1.96* (dv_by_years_final['доля ДН (%)']/100*(1-dv_by_years_final['доля ДН (%)']/100)/dv_by_years_final['число женских дел'])**(1/2))
dv_by_years_final['доля ДН (верхняя граница)'] = 100*(dv_by_years_final['доля ДН (%)']/100 + 1.96* (dv_by_years_final['доля ДН (%)']/100*(1-dv_by_years_final['доля ДН (%)']/100)/dv_by_years_final['число женских дел'])**(1/2))
dv_by_years_final['доля партнерского (нижняя граница)'] = 100*(dv_by_years_final['доля партнерского из ДН (%)']/100 - 1.96* (dv_by_years_final['доля партнерского из ДН (%)']/100*(1-dv_by_years_final['доля партнерского из ДН (%)']/100)/dv_by_years_final['из них ДН (с поправкой на точность)'])**(1/2))
dv_by_years_final['доля партнерского (верхняя граница)'] = 100*(dv_by_years_final['доля партнерского из ДН (%)']/100 + 1.96* (dv_by_years_final['доля партнерского из ДН (%)']/100*(1-dv_by_years_final['доля партнерского из ДН (%)']/100)/dv_by_years_final['из них ДН (с поправкой на точность)'])**(1/2))
dv_by_years_final

,год,число женских дел,из них ДН,из них ДН (с поправкой на точность),из них партнерское,из них партнерское (с поправкой на точность),доля ДН (%),доля партнерского из ДН (%),доля ДН (нижняя граница),доля ДН (верхняя граница),доля партнерского (нижняя граница),доля партнерского (верхняя граница)
0,2011,2075,1578,1357.08,1356.0,1096.1904,65.401446,80.775665,63.354673,67.448219,78.679048,82.872283
1,2019,1419,1108,952.88,965.0,780.1060,67.151515,81.868231,64.707801,69.595230,79.421903,84.314559


In [13]:
#dv_by_years_final.to_excel('Динамика_ДН_по_годам_2011_2019.xlsx')

#### Шаг 4. Добавление регионов

In [123]:
years = list(range(2011, 2020))
regions = set()

for year in years:
    if year == 2018:
        continue
    texts = pd.read_csv(f'Данные_за_все_года/{year}_women.csv', index_col=0)
    regions.update(set(texts['region'].replace({'Москва': 'город Москва', 'Город Москва': 'город Москва'}).drop_duplicates()))
len(regions)

84

In [124]:
# без поправки на точность

import pandas as pd

years = list(range(2011, 2020))
regions_dynamics = pd.DataFrame(index=regions)

for year in years:
    if year == 2018:
        continue
    texts = pd.read_csv(f'Данные_за_все_года/{year}_women.csv', index_col=0)
    texts['region'] = texts['region'].replace({'Москва': 'город Москва', 'Город Москва': 'город Москва'})
    predictions = pd.read_excel(f'Данные_за_все_года/{year}_women_predictions.xlsx', index_col=0)
    texts[['DV', 'IPV']] = predictions[['DV', 'IPV']]
    #print(year, len(texts), len(predictions))
    #regions_by_year = texts.groupby('region')['DV','IPV'].sum()
    regions_by_year = pd.pivot_table(texts, index='region', values=['DV','IPV'], aggfunc=sum)
    regions_dynamics[[f'DV_{year}', f'IPV_{year}']] = regions_by_year[['DV','IPV']]
    #print(regions_by_year[['DV','IPV']].sum())
    #print(regions_dynamics[[f'DV_{year}', f'IPV_{year}']].sum())
    regions_dynamics[[f'DV_{year}', f'IPV_{year}']] = regions_dynamics[[f'DV_{year}', f'IPV_{year}']].fillna(0)
    
    display(regions_dynamics)

,DV_2011,IPV_2011
Иркутская область,27.0,21.0
Рязанская область,6.0,4.0
Курская область,17.0,14.0
Республика Бурятия,32.0,26.0
Ростовская область,21.0,17.0
...,...,...
Республика Карелия,10.0,9.0
город Санкт-Петербург,13.0,13.0
Саратовская область,25.0,24.0
Кировская область,8.0,6.0


,DV_2011,IPV_2011,DV_2012,IPV_2012
Иркутская область,27.0,21.0,31.0,25.0
Рязанская область,6.0,4.0,6.0,6.0
Курская область,17.0,14.0,10.0,8.0
Республика Бурятия,32.0,26.0,34.0,27.0
Ростовская область,21.0,17.0,16.0,15.0
...,...,...,...,...
Республика Карелия,10.0,9.0,7.0,6.0
город Санкт-Петербург,13.0,13.0,13.0,12.0
Саратовская область,25.0,24.0,38.0,35.0
Кировская область,8.0,6.0,20.0,17.0


,DV_2011,IPV_2011,DV_2012,IPV_2012,DV_2013,IPV_2013
Иркутская область,27.0,21.0,31.0,25.0,40.0,34.0
Рязанская область,6.0,4.0,6.0,6.0,4.0,3.0
Курская область,17.0,14.0,10.0,8.0,17.0,14.0
Республика Бурятия,32.0,26.0,34.0,27.0,25.0,19.0
Ростовская область,21.0,17.0,16.0,15.0,31.0,27.0
...,...,...,...,...,...,...
Республика Карелия,10.0,9.0,7.0,6.0,5.0,4.0
город Санкт-Петербург,13.0,13.0,13.0,12.0,14.0,11.0
Саратовская область,25.0,24.0,38.0,35.0,35.0,32.0
Кировская область,8.0,6.0,20.0,17.0,16.0,12.0


,DV_2011,IPV_2011,DV_2012,IPV_2012,DV_2013,IPV_2013,DV_2014,IPV_2014
Иркутская область,27.0,21.0,31.0,25.0,40.0,34.0,50.0,43.0
Рязанская область,6.0,4.0,6.0,6.0,4.0,3.0,8.0,7.0
Курская область,17.0,14.0,10.0,8.0,17.0,14.0,14.0,13.0
Республика Бурятия,32.0,26.0,34.0,27.0,25.0,19.0,28.0,25.0
Ростовская область,21.0,17.0,16.0,15.0,31.0,27.0,26.0,20.0
...,...,...,...,...,...,...,...,...
Республика Карелия,10.0,9.0,7.0,6.0,5.0,4.0,16.0,14.0
город Санкт-Петербург,13.0,13.0,13.0,12.0,14.0,11.0,18.0,18.0
Саратовская область,25.0,24.0,38.0,35.0,35.0,32.0,44.0,40.0
Кировская область,8.0,6.0,20.0,17.0,16.0,12.0,30.0,28.0


,DV_2011,IPV_2011,DV_2012,IPV_2012,DV_2013,IPV_2013,DV_2014,IPV_2014,DV_2015,IPV_2015
Иркутская область,27.0,21.0,31.0,25.0,40.0,34.0,50.0,43.0,52,40.0
Рязанская область,6.0,4.0,6.0,6.0,4.0,3.0,8.0,7.0,5,5.0
Курская область,17.0,14.0,10.0,8.0,17.0,14.0,14.0,13.0,10,8.0
Республика Бурятия,32.0,26.0,34.0,27.0,25.0,19.0,28.0,25.0,30,26.0
Ростовская область,21.0,17.0,16.0,15.0,31.0,27.0,26.0,20.0,38,32.0
...,...,...,...,...,...,...,...,...,...,...
Республика Карелия,10.0,9.0,7.0,6.0,5.0,4.0,16.0,14.0,11,10.0
город Санкт-Петербург,13.0,13.0,13.0,12.0,14.0,11.0,18.0,18.0,19,19.0
Саратовская область,25.0,24.0,38.0,35.0,35.0,32.0,44.0,40.0,29,26.0
Кировская область,8.0,6.0,20.0,17.0,16.0,12.0,30.0,28.0,22,20.0


,DV_2011,IPV_2011,DV_2012,IPV_2012,DV_2013,IPV_2013,DV_2014,IPV_2014,DV_2015,IPV_2015,DV_2016,IPV_2016
Иркутская область,27.0,21.0,31.0,25.0,40.0,34.0,50.0,43.0,52,40.0,43,35.0
Рязанская область,6.0,4.0,6.0,6.0,4.0,3.0,8.0,7.0,5,5.0,5,5.0
Курская область,17.0,14.0,10.0,8.0,17.0,14.0,14.0,13.0,10,8.0,10,8.0
Республика Бурятия,32.0,26.0,34.0,27.0,25.0,19.0,28.0,25.0,30,26.0,29,23.0
Ростовская область,21.0,17.0,16.0,15.0,31.0,27.0,26.0,20.0,38,32.0,25,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Республика Карелия,10.0,9.0,7.0,6.0,5.0,4.0,16.0,14.0,11,10.0,6,6.0
город Санкт-Петербург,13.0,13.0,13.0,12.0,14.0,11.0,18.0,18.0,19,19.0,19,17.0
Саратовская область,25.0,24.0,38.0,35.0,35.0,32.0,44.0,40.0,29,26.0,22,19.0
Кировская область,8.0,6.0,20.0,17.0,16.0,12.0,30.0,28.0,22,20.0,16,13.0


,DV_2011,IPV_2011,DV_2012,IPV_2012,DV_2013,IPV_2013,DV_2014,IPV_2014,DV_2015,IPV_2015,DV_2016,IPV_2016,DV_2017,IPV_2017
Иркутская область,27.0,21.0,31.0,25.0,40.0,34.0,50.0,43.0,52,40.0,43,35.0,46.0,34.0
Рязанская область,6.0,4.0,6.0,6.0,4.0,3.0,8.0,7.0,5,5.0,5,5.0,5.0,5.0
Курская область,17.0,14.0,10.0,8.0,17.0,14.0,14.0,13.0,10,8.0,10,8.0,5.0,3.0
Республика Бурятия,32.0,26.0,34.0,27.0,25.0,19.0,28.0,25.0,30,26.0,29,23.0,33.0,32.0
Ростовская область,21.0,17.0,16.0,15.0,31.0,27.0,26.0,20.0,38,32.0,25,24.0,19.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Республика Карелия,10.0,9.0,7.0,6.0,5.0,4.0,16.0,14.0,11,10.0,6,6.0,9.0,9.0
город Санкт-Петербург,13.0,13.0,13.0,12.0,14.0,11.0,18.0,18.0,19,19.0,19,17.0,29.0,28.0
Саратовская область,25.0,24.0,38.0,35.0,35.0,32.0,44.0,40.0,29,26.0,22,19.0,19.0,19.0
Кировская область,8.0,6.0,20.0,17.0,16.0,12.0,30.0,28.0,22,20.0,16,13.0,16.0,16.0


,DV_2011,IPV_2011,DV_2012,IPV_2012,DV_2013,IPV_2013,DV_2014,IPV_2014,DV_2015,IPV_2015,DV_2016,IPV_2016,DV_2017,IPV_2017,DV_2019,IPV_2019
Иркутская область,27.0,21.0,31.0,25.0,40.0,34.0,50.0,43.0,52,40.0,43,35.0,46.0,34.0,30.0,24.0
Рязанская область,6.0,4.0,6.0,6.0,4.0,3.0,8.0,7.0,5,5.0,5,5.0,5.0,5.0,9.0,2.0
Курская область,17.0,14.0,10.0,8.0,17.0,14.0,14.0,13.0,10,8.0,10,8.0,5.0,3.0,6.0,6.0
Республика Бурятия,32.0,26.0,34.0,27.0,25.0,19.0,28.0,25.0,30,26.0,29,23.0,33.0,32.0,21.0,17.0
Ростовская область,21.0,17.0,16.0,15.0,31.0,27.0,26.0,20.0,38,32.0,25,24.0,19.0,16.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Республика Карелия,10.0,9.0,7.0,6.0,5.0,4.0,16.0,14.0,11,10.0,6,6.0,9.0,9.0,6.0,6.0
город Санкт-Петербург,13.0,13.0,13.0,12.0,14.0,11.0,18.0,18.0,19,19.0,19,17.0,29.0,28.0,9.0,8.0
Саратовская область,25.0,24.0,38.0,35.0,35.0,32.0,44.0,40.0,29,26.0,22,19.0,19.0,19.0,20.0,15.0
Кировская область,8.0,6.0,20.0,17.0,16.0,12.0,30.0,28.0,22,20.0,16,13.0,16.0,16.0,21.0,18.0


#### Шаг 5. Добавляем общее число дел

In [87]:
years = list(range(2011, 2020))
n_cases = []
for year in years:
    if year == 2018:
        continue
    merged = pd.read_csv(f'Данные_за_все_года/{year}_merged.csv', index_col=0)
    n_cases.append([year, len(merged)])
n_cases

[[2011, 8914],
 [2012, 8757],
 [2013, 9323],
 [2014, 10652],
 [2015, 10036],
 [2016, 9835],
 [2017, 9262],
 [2019, 6462]]

In [88]:
pd.DataFrame(n_cases, columns=['year', 'Количество дел']).to_excel("Количество дел.xlsx")

#### Шаг 6. Добавляем инфу о 2018 годе

In [125]:
data_2018 = pd.read_excel('Только_женские_тексты_с_разметкой_предобр_без_98.xlsx', index_col=0)
data_2018['IPV'] = data_2018['IPV'].replace({2: 0})
data_2018.head()

,text,DV,IPV,text_prep
ID,,,,
82060183bdf7472e85540c4370d3693c,Дело № 1-318/2018\nПоступило в суд 04.12.2018\...,1,1.0,дело поступать российский федерация декабрь ку...
377054bb5594776bb30ff64c6ebcffb8,Уг.д. № 1-45/2018 (11701330019047312)\nП Р И Г...,1,1.0,российский федерация город яранск кировский об...
6df614cdfd7d029cde776d52ed7932d2,UNKNOWN {\n\tTEXT-ALIGN: justify\n}\n\n\n\n ...,1,1.0,дело российский федерация урюпинск август судь...
50593bc68988c7aae8f9f2f2bf243f8a,Дело № 1-19/2018\nПриговор \nИменем Российской...,1,1.0,дело приговор российский федерация тихвин янва...
54e009a196b97005a1b4d828670345e7,Дело №1-1-33\2018\nПРИГОВОР\nИменем Российской...,1,1.0,дело приговор российский федерация март город ...


In [126]:
display(data_2018['DV'].value_counts(normalize=True), data_2018['IPV'].value_counts(normalize=True))

1    0.691176
0    0.308824
Name: DV, dtype: float64

1.0    0.80213
0.0    0.19787
Name: IPV, dtype: float64

In [127]:
len(data_2018)

1632

In [128]:
data_reg_2018 = pd.read_excel('2018_women_cases_MERGED.xlsx', index_col=0).drop_duplicates('ID').set_index("ID")
data_reg_2018

,Номер дела (материала),Стороны,Результат,Судья,Статья УК РФ,URL,URL_verdict,text,Источник,Субъект РФ,Тип документа,ФИО,Наименование суда,№,Аннотация,Дата поступления,Дата решения
ID,,,,,,,,,,,,,,,,,
c21b02c4-9fcd-4375-b0f6-2de4aa87b300,01-0796/2018,"Подсудимый: Васильев Н.Б. (Ст. 105, Ч. 1;)","Вступило в силу, 23.10.2018",Борисенкова Н.В.,"Ст. 105, Ч. 1",https://mos-gorsud.ru/rs/nagatinskij/services/...,https://mos-gorsud.ru/rs/nagatinskij/cases/doc...,1-796/2018 ПРИГОВОР Именем Российской Федераци...,Мосгорсуд,Москва,Приговор,Васильев Н.Б.,Нагатинский районный суд,NaN,NaN,NaN,NaN
2f8ccdca-bf56-4dda-a8ab-13791848f3e6,01-0520/2018,"Подсудимый: Родимушкин Н.В. (Ст. 105, Ч. 1;)","Изменено, 06.02.2019",Филатов А.Ю.,"Ст. 105, Ч. 1",https://mos-gorsud.ru/rs/nagatinskij/services/...,https://mos-gorsud.ru/rs/nagatinskij/cases/doc...,Уголовное дело № 1-520/2018 П Р И Г О В О Р Им...,Мосгорсуд,Москва,Приговор,Родимушкин Н.В.,Нагатинский районный суд,NaN,NaN,NaN,NaN
a3bd299f-e55e-474d-84f0-7cdcc7f53869,01-0511/2018,"Подсудимый: Семенов Л.В. (Ст. 105, Ч. 1;)","Вступило в силу, 21.12.2018",Чечко Л.Н.,"Ст. 105, Ч. 1",https://mos-gorsud.ru/rs/zyuzinskij/services/c...,https://mos-gorsud.ru/rs/zyuzinskij/cases/docs...,\t\t\t\t\t\t\t ...,Мосгорсуд,Москва,Приговор,Семенов Л.В.,Зюзинский районный суд,NaN,NaN,NaN,NaN
e26be2b3-cd2d-4237-a202-6ccc720ad557,01-0472/2018,"Подсудимый: Остриков А.В. (Ст. 105, Ч. 1;)","Вступило в силу, 17.07.2018",Борисенкова Н.В.,"Ст. 105, Ч. 1",https://mos-gorsud.ru/rs/nagatinskij/services/...,https://mos-gorsud.ru/rs/nagatinskij/cases/doc...,1-472/2018 ПРИГОВОР Именем Российской Федераци...,Мосгорсуд,Москва,Приговор,Остриков А.В.,Нагатинский районный суд,NaN,NaN,NaN,NaN
743790b9-307e-49e4-bb69-1a4b4e3fd872,01-0471/2018,"Подсудимый: Никишин П.О. (Ст. 105, Ч. 2, пп. ...","Вступило в силу, 25.10.2018",Кузнецов В.С.,"Ст. 105, Ч. 2",https://mos-gorsud.ru/rs/lyublinskij/services/...,https://mos-gorsud.ru/rs/lyublinskij/cases/doc...,Дело № 1-471/2018 П Р И Г О В ...,Мосгорсуд,Москва,Приговор,Никишин П.О.,Люблинский районный суд,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552c2eac9cc2672729ab438742c61bf3,1-264/2018,NaN,Вынесен ПРИГОВОР,NaN,Статья 166 Часть 1; Статья 158 Часть 2 п.в; Ст...,NaN,NaN,Дело № 1-264/18П Р И Г О В О Р Именем Российск...,ГАС Правосудие,Алтайский край,Приговор,Сурсков Андрей Александрович,Октябрьский районный суд г. Барнаула,3153.0,... я 166 Часть 1; Статья 158 Часть 2 п.в; Ста...,08.05.2018,25.06.2018
6f42bca4eb76fcd6acdfcf216edb1294,1-598/2018,NaN,Вынесен ПРИГОВОР,NaN,Статья 161 Часть 1; Статья 158 Часть 2 п.в; Ст...,NaN,NaN,Дело № (сл. №)\n\nП Р И Г О В О Р \nИ М Е Н Е ...,ГАС Правосудие,Московская область,Приговор,Шарашкин Б.А.,Орехово-Зуевский городской суд,3154.0,... 8 Часть 2 п.в; Статья 158 Часть 3 п.а; Ста...,30.08.2018,08.11.2018
ce8b099236e8ae205ad95bc76e9bda54,1-20/2018 (1-364/2017;),NaN,Вынесен ПРИГОВОР,NaN,Статья 115 Часть 2 п.в; Статья 115 Часть 2 п.в...,NaN,NaN,UNKNOWN {\r\n\tTEXT-ALIGN: justify\r\n}\r\n\n\...,ГАС Правосудие,Омская область,Приговор,"Воякин Сергей Николаевич, Хибарин Руслан Анато...",Куйбышевский районный суд г. Омска,3159.0,"... сть 2 п.в; Статья 127 Часть 2 п.п.г,ж; Ста...",14.09.2017,09.01.2018


In [129]:
data_2018['IPV'].value_counts()

1.0    904
0.0    223
Name: IPV, dtype: int64

In [130]:
data_2018['region'] = data_reg_2018['Субъект РФ'].replace({'Москва': 'город Москва', 'Город Москва': 'город Москва'})
data_2018.head()

,text,DV,IPV,text_prep,region
ID,,,,,
82060183bdf7472e85540c4370d3693c,Дело № 1-318/2018\nПоступило в суд 04.12.2018\...,1,1.0,дело поступать российский федерация декабрь ку...,Новосибирская область
377054bb5594776bb30ff64c6ebcffb8,Уг.д. № 1-45/2018 (11701330019047312)\nП Р И Г...,1,1.0,российский федерация город яранск кировский об...,Кировская область
6df614cdfd7d029cde776d52ed7932d2,UNKNOWN {\n\tTEXT-ALIGN: justify\n}\n\n\n\n ...,1,1.0,дело российский федерация урюпинск август судь...,Волгоградская область
50593bc68988c7aae8f9f2f2bf243f8a,Дело № 1-19/2018\nПриговор \nИменем Российской...,1,1.0,дело приговор российский федерация тихвин янва...,Ленинградская область
54e009a196b97005a1b4d828670345e7,Дело №1-1-33\2018\nПРИГОВОР\nИменем Российской...,1,1.0,дело приговор российский федерация март город ...,Орловская область


In [131]:
regions_2018 = pd.pivot_table(data_2018, index='region', values=['DV','IPV'], aggfunc=sum)
regions_2018.head()

,DV,IPV
region,,
Алтайский край,32,25.0
Амурская область,10,7.0
Архангельская область,14,8.0
Астраханская область,7,5.0
Белгородская область,5,4.0


In [132]:
regions_dynamics[['DV_2018', 'IPV_2018']] = regions_2018[['DV', 'IPV']]
regions_dynamics.tail()

,DV_2011,IPV_2011,DV_2012,IPV_2012,DV_2013,IPV_2013,DV_2014,IPV_2014,DV_2015,IPV_2015,DV_2016,IPV_2016,DV_2017,IPV_2017,DV_2019,IPV_2019,DV_2018,IPV_2018
Республика Карелия,10.0,9.0,7.0,6.0,5.0,4.0,16.0,14.0,11,10.0,6,6.0,9.0,9.0,6.0,6.0,11.0,10.0
город Санкт-Петербург,13.0,13.0,13.0,12.0,14.0,11.0,18.0,18.0,19,19.0,19,17.0,29.0,28.0,9.0,8.0,11.0,8.0
Саратовская область,25.0,24.0,38.0,35.0,35.0,32.0,44.0,40.0,29,26.0,22,19.0,19.0,19.0,20.0,15.0,9.0,7.0
Кировская область,8.0,6.0,20.0,17.0,16.0,12.0,30.0,28.0,22,20.0,16,13.0,16.0,16.0,21.0,18.0,13.0,9.0
Республика Мордовия,5.0,5.0,7.0,7.0,12.0,10.0,7.0,6.0,7,6.0,7,5.0,2.0,1.0,5.0,5.0,2.0,2.0


In [133]:
regions_dynamics = regions_dynamics[[f'DV_{year}' for year in range(2011, 2020)] + [f'IPV_{year}' for year in range(2011, 2020)]]
regions_dynamics.head()

,DV_2011,DV_2012,DV_2013,DV_2014,DV_2015,DV_2016,DV_2017,DV_2018,DV_2019,IPV_2011,IPV_2012,IPV_2013,IPV_2014,IPV_2015,IPV_2016,IPV_2017,IPV_2018,IPV_2019
Иркутская область,27.0,31.0,40.0,50.0,52,43,46.0,30.0,30.0,21.0,25.0,34.0,43.0,40.0,35.0,34.0,21.0,24.0
Рязанская область,6.0,6.0,4.0,8.0,5,5,5.0,5.0,9.0,4.0,6.0,3.0,7.0,5.0,5.0,5.0,4.0,2.0
Курская область,17.0,10.0,17.0,14.0,10,10,5.0,8.0,6.0,14.0,8.0,14.0,13.0,8.0,8.0,3.0,3.0,6.0
Республика Бурятия,32.0,34.0,25.0,28.0,30,29,33.0,30.0,21.0,26.0,27.0,19.0,25.0,26.0,23.0,32.0,24.0,17.0
Ростовская область,21.0,16.0,31.0,26.0,38,25,19.0,18.0,8.0,17.0,15.0,27.0,20.0,32.0,24.0,16.0,16.0,8.0


In [134]:
regions_dynamics.loc['Всего'] = regions_dynamics.sum()
regions_dynamics['DV_Всего'] = regions_dynamics[[col for col in regions_dynamics.columns if 'DV' in col]].sum(axis=1)
regions_dynamics['IPV_Всего'] = regions_dynamics[[col for col in regions_dynamics.columns if 'IPV' in col]].sum(axis=1)
regions_dynamics.tail()

,DV_2011,DV_2012,DV_2013,DV_2014,DV_2015,DV_2016,DV_2017,DV_2018,DV_2019,IPV_2011,IPV_2012,IPV_2013,IPV_2014,IPV_2015,IPV_2016,IPV_2017,IPV_2018,IPV_2019,DV_Всего,IPV_Всего
город Санкт-Петербург,13.0,13.0,14.0,18.0,19.0,19.0,29.0,11.0,9.0,13.0,12.0,11.0,18.0,19.0,17.0,28.0,8.0,8.0,145.0,134.0
Саратовская область,25.0,38.0,35.0,44.0,29.0,22.0,19.0,9.0,20.0,24.0,35.0,32.0,40.0,26.0,19.0,19.0,7.0,15.0,241.0,217.0
Кировская область,8.0,20.0,16.0,30.0,22.0,16.0,16.0,13.0,21.0,6.0,17.0,12.0,28.0,20.0,13.0,16.0,9.0,18.0,162.0,139.0
Республика Мордовия,5.0,7.0,12.0,7.0,7.0,7.0,2.0,2.0,5.0,5.0,7.0,10.0,6.0,6.0,5.0,1.0,2.0,5.0,54.0,47.0
Всего,1578.0,1573.0,1654.0,1915.0,1742.0,1734.0,1581.0,1127.0,1108.0,1356.0,1345.0,1416.0,1652.0,1504.0,1487.0,1364.0,904.0,965.0,14012.0,11993.0


In [135]:
len(regions_dynamics)

85

In [136]:
# без поправки на точность
regions_dynamics.to_excel('Динамика по регионам (без поправки).xlsx')

In [137]:
regions_dynamics.drop('Всего', inplace = True)
regions_dynamics.drop(['DV_Всего', 'IPV_Всего'], axis=1, inplace=True)

for col in regions_dynamics.columns:
    if '2018' in col:
        continue
    if 'DV' in col:
        regions_dynamics[col] = regions_dynamics[col]*prec_dv_model
    elif 'IPV' in col:
        regions_dynamics[col] = regions_dynamics[col]*prec_dv_model*prec_ipv_model
        
regions_dynamics.loc['Всего'] = regions_dynamics.sum()
regions_dynamics['DV_Всего'] = regions_dynamics[[col for col in regions_dynamics.columns if 'DV' in col]].sum(axis=1)
regions_dynamics['IPV_Всего'] = regions_dynamics[[col for col in regions_dynamics.columns if 'IPV' in col]].sum(axis=1)
regions_dynamics.tail()

,DV_2011,DV_2012,DV_2013,DV_2014,DV_2015,DV_2016,DV_2017,DV_2018,DV_2019,IPV_2011,IPV_2012,IPV_2013,IPV_2014,IPV_2015,IPV_2016,IPV_2017,IPV_2018,IPV_2019,DV_Всего,IPV_Всего
город Санкт-Петербург,11.18,11.18,12.04,15.48,16.34,16.34,24.94,11.0,7.74,10.5092,9.7008,8.8924,14.5512,15.3596,13.7428,22.6352,8.0,6.4672,126.24,109.8584
Саратовская область,21.50,32.68,30.10,37.84,24.94,18.92,16.34,9.0,17.20,19.4016,28.2940,25.8688,32.3360,21.0184,15.3596,15.3596,7.0,12.1260,208.52,176.7640
Кировская область,6.88,17.20,13.76,25.80,18.92,13.76,13.76,13.0,18.06,4.8504,13.7428,9.7008,22.6352,16.1680,10.5092,12.9344,9.0,14.5512,141.14,114.0920
Республика Мордовия,4.30,6.02,10.32,6.02,6.02,6.02,1.72,2.0,4.30,4.0420,5.6588,8.0840,4.8504,4.8504,4.0420,0.8084,2.0,4.0420,46.72,38.3780
Всего,1357.08,1352.78,1422.44,1646.90,1498.12,1491.24,1359.66,1127.0,952.88,1096.1904,1087.2980,1144.6944,1335.4768,1215.8336,1202.0908,1102.6576,904.0,780.1060,12208.10,9868.3476


In [139]:
regions_dynamics.apply(round).to_excel('Динамика по регионам (с поправкой).xlsx')

#### Шаг 7. Ключевые слова

In [1]:
import pickle

with open("cvect.pkl", 'rb') as file:
    cvect = pickle.load(file)

In [2]:
cvect.get_feature_names()

['абонентский',
 'абонентский номер',
 'август',
 'автомашина',
 'автомобиль',
 'агрессивный',
 'агрессия',
 'адвокат',
 'адвокатский',
 'адекватно',
 'адекватный',
 'административный',
 'административный ответственность',
 'администрация',
 'акт',
 'акт медицинский',
 'активно',
 'активный',
 'активный действие',
 'активный сопротивление',
 'активный способствование',
 'активный целенаправленный',
 'алкоголизм',
 'алкоголь',
 'алкоголь поскольку',
 'алкогольный',
 'алкогольный напиток',
 'алкогольный опьянение',
 'амбулаторный',
 'амбулаторный комплексный',
 'аморальность',
 'аморальный',
 'аморальный поведение',
 'анализ',
 'анализировать',
 'аналогичный',
 'анатомический',
 'антиген',
 'апелляционный',
 'апелляционный жалоба',
 'апелляционный инстанция',
 'апелляционный порядок',
 'апелляционный представление',
 'апрель',
 'арест',
 'артерия',
 'асфиксия',
 'аффект',
 'бабушка',
 'балкон',
 'банка',
 'банковский',
 'банковский карта',
 'баня',
 'бегать',
 'бедро',
 'бежать',
 'белый

In [3]:
with open("gbc_dv.pkl", 'rb') as file:
    dv = pickle.load(file)
    
with open("gbc_ipv.pkl", 'rb') as file:
    ipv = pickle.load(file)

In [12]:
import pandas as pd
pd.DataFrame(zip(cvect.get_feature_names(), list(dv.feature_importances_)), columns=['token', 'imp']).sort_values('imp', ascending=False).to_excel('words_dv.xlsx')

In [13]:
pd.DataFrame(zip(cvect.get_feature_names(), list(ipv.feature_importances_)), columns=['token', 'imp']).sort_values('imp', ascending=False).to_excel('words_ipv.xlsx')

Поймём направление прогноза с помощью более простых моделей

In [18]:
import pandas as pd
dv_words = pd.read_excel('words_dv.xlsx', index_col=0).set_index('token')
dv_words.head()

,imp
token,
подсудимая,0.045905
ревность,0.044099
хищение,0.042886
скандал,0.025388
ссора,0.024234


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, f1_score

data = pd.read_excel('preprocessed_data_2018.xlsx', index_col=0).set_index('ID')
cvect = CountVectorizer(min_df=0.05, ngram_range=(1, 2)).fit(data['text_prep'])
matrix = cvect.transform(data['text_prep'])
td_matrix = pd.DataFrame(matrix.toarray(), index=data.index, columns=cvect.get_feature_names())
td_matrix.head()

,абонентский,абонентский номер,август,автомашина,автомобиль,агрессивный,агрессия,адвокат,адвокатский,адекватно,...,являться причина,являться родной,явный,ягодица,язык,якобы,январь,яремный,яремный вена,ящик
ID,,,,,,,,,,,,,,,,,,,,,
82060183bdf7472e85540c4370d3693c,0,0,0,0,0,1,0,2,0,0,...,0,1,0,0,0,0,0,0,0,0
377054bb5594776bb30ff64c6ebcffb8,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
6df614cdfd7d029cde776d52ed7932d2,0,0,1,0,0,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
50593bc68988c7aae8f9f2f2bf243f8a,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,3,0,0,0
54e009a196b97005a1b4d828670345e7,0,0,0,0,5,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
td_matrix['DV'] = data['DV']
td_matrix['IPV'] = data['IPV']

y_dv = td_matrix['DV'].astype(int)
X_dv = td_matrix.drop(["DV", 'IPV'], axis=1)

y_ipv = td_matrix[td_matrix['IPV'].notna()]['IPV'].astype(int).replace({2: 0}) # 0 - родственник, 1 - партнер
X_ipv = td_matrix[td_matrix['IPV'].notna()].drop(["DV", 'IPV'], axis=1)
print(y_dv.shape, X_dv.shape, y_ipv.shape, X_ipv.shape)

(1632,) (1632, 5522) (1128,) (1128, 5522)


Модель DV

In [8]:
clf = LogisticRegression(random_state=42, max_iter=10000).fit(X_dv, y_dv)

In [16]:
coefs_dv = pd.DataFrame(zip(cvect.get_feature_names(), list(clf.coef_[0])), columns=['token', 'coef']).set_index('token')
coefs_dv.head()

,coef
token,
абонентский,0.032543
абонентский номер,-0.002778
август,0.053216
автомашина,0.064584
автомобиль,0.013338


In [25]:
dv_words['coef'] = coefs_dv['coef']
dv_words['predict'] = dv_words['coef'].apply(lambda x: 'прогноз - ДН' if x > 0 else 'прогноз - НЕ ДН')
dv_words.head()

,imp,coef,predict
token,,,
подсудимая,0.045905,-0.270593,прогноз - НЕ ДН
ревность,0.044099,0.370185,прогноз - ДН
хищение,0.042886,-0.154486,прогноз - НЕ ДН
скандал,0.025388,0.242490,прогноз - ДН
ссора,0.024234,0.344314,прогноз - ДН


In [26]:
#dv_words.to_excel('words_dv.xlsx')

In [27]:
ipv_words = pd.read_excel('words_ipv.xlsx', index_col=0).set_index('token')
ipv_words.head()

,imp
token,
мать,0.333776
подсудимая,0.072365
ревность,0.070919
бабушка,0.063138
сожительница,0.041377


In [29]:
clf = LogisticRegression(random_state=42, max_iter=10000).fit(X_ipv, y_ipv)

In [30]:
coefs_ipv = pd.DataFrame(zip(cvect.get_feature_names(), list(clf.coef_[0])), columns=['token', 'coef']).set_index('token')
coefs_ipv.head()

,coef
token,
абонентский,0.008986
абонентский номер,0.029857
август,-0.121702
автомашина,0.012726
автомобиль,0.027747


In [31]:
ipv_words['coef'] = coefs_ipv['coef']
ipv_words['predict'] = ipv_words['coef'].apply(lambda x: 'прогноз - ПАРТНЕР' if x > 0 else 'прогноз - РОДСТВЕННИК')
ipv_words.head()

,imp,coef,predict
token,,,
мать,0.333776,-0.361695,прогноз - РОДСТВЕННИК
подсудимая,0.072365,-0.298342,прогноз - РОДСТВЕННИК
ревность,0.070919,0.213757,прогноз - ПАРТНЕР
бабушка,0.063138,-0.214108,прогноз - РОДСТВЕННИК
сожительница,0.041377,0.319381,прогноз - ПАРТНЕР


In [32]:
#ipv_words.to_excel('words_ipv.xlsx')

In [ ]:
print(classification_report(y_train, gbc.predict(X_train)))
print(classification_report(y_test, gbc.predict(X_test))